In [45]:
import argparse
import random
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np
import torch
from config import get_config
from dataset import data_loader
from neural_methods import trainer
from neural_methods.trainer.BaseTrainer import BaseTrainer
from unsupervised_methods.unsupervised_predictor import unsupervised_predict
from torch.utils.data import DataLoader

In [46]:
parser = argparse.ArgumentParser()
parser = add_args(parser)
parser = BaseTrainer.add_trainer_args(parser)
parser = data_loader.BaseLoader.BaseLoader.add_data_loader_args(parser)
args = parser.parse_args(args=["--config_file", "configs/train_configs/CogPhys_PHYSFORMER_BASIC.yaml"])


# configurations.
config = get_config(args)
print('Configuration:', config)

=> Merging a config file from configs/train_configs/CogPhys_PHYSFORMER_BASIC.yaml
Configuration: BASE: ['']
DEVICE: cuda:0
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: FFT
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False
    WINDOW_SIZE: 30
  MODEL_PATH: 
LOG:
  PATH: runs/exp
MODEL:
  BIGSMALL:
    FRAME_DEPTH: 3
  CONTRASTPHYS:
    CHANNELS: 3
    FRAME_NUM: 160
    S: 2
  DROP_RATE: 0.1
  EFFICIENTPHYS:
    FRAME_DEPTH: 10
  FactorizePhys:
    CHANNELS: 3
    FRAME_NUM: 160
    MD_FSAM: False
    MD_INFERENCE: True
    MD_R: 1
    MD_RESIDUAL: True
    MD_S: 1
    MD_STEPS: 4
    MD_TRANSFORM: T_KAB
    MD_TYPE: NMF
    TYPE: Standard
  MODEL_DIR: runs/exp/RGB_PhysFormer/PreTrainedModels
  NAME: PhysFormer
  PHYSFORMER:
    DIM: 96
    FF_DIM: 144
    NUM_HEADS: 4
    NUM_LAYERS: 12
    PATCH_SIZE: 4
    THETA: 0.7
  PHYSNET:
    FRAME_NUM: 64
  RESUME: 
  TSCAN:
    FRAME_DEPTH: 10
  iBVPNet:
    CHANNELS: 3
    FRAME_NUM: 160
NUM_OF_GPU_TRAIN: 1
TEST:
  DATA:
    BEGIN: 0.0
 

In [47]:
train_loader = data_loader.CogPhysLoader.CogPhysLoader
data_loader_dict = dict()


if (config.TRAIN.DATA.DATASET and config.TRAIN.DATA.DATA_PATH):
    train_data_loader = train_loader(
        name="train",
        data_path=config.TRAIN.DATA.DATA_PATH,
        config_data=config.TRAIN.DATA,
        device=config.DEVICE)
    data_loader_dict['train'] = DataLoader(
        dataset=train_data_loader,
        num_workers=0,
        batch_size=config.TRAIN.BATCH_SIZE,
        shuffle=True,
        worker_init_fn=seed_worker,
        generator=train_generator
    )
else:
    data_loader_dict['train'] = None
    

Cached Data Path /shared/ab227/CogPhys/chunked_dataset

Data Path /shared/ab227/CogPhys/chunked_dataset

train Preprocessed Dataset Length: 1776



In [48]:
#Error at ('v28_pattern_rest', 'v16_still') ('6', '8')

#get index of v_28 and chunk 6
for idx in range(len(train_data_loader)):
    sample = train_data_loader[idx] 
    vid_name = sample[2]
    chunk_idx = sample[3]
    if vid_name == 'v28_pattern_rest' and chunk_idx == '6':
        print("Found at index:", idx)
        # You can now save or process this sample
        rgb_tensor = sample[0][:,0,:,:].permute(1,2,0)  # shape [3,128,128] -> [128,128,3]
        ppg_signal = sample[1]
        
        #saving
        np.save('temp1_ppg.npy', ppg_signal.cpu().numpy())
        np.save('temp1_rgb.npy', rgb_tensor.cpu().numpy())

        #plotting
        plt.imsave('temp_frame1.png', rgb_tensor.cpu().numpy())
        plt.clf()
        plt.plot(ppg_signal)
        plt.savefig('temp1_ppg')
        break
  
#get index of v16 and chunk 8  
for idx in range(len(train_data_loader)):
    sample = train_data_loader[idx] #1268 is the index of v28_...
    vid_name = sample[2]
    chunk_idx = sample[3]
    if vid_name == 'v16_still' and chunk_idx == '8':
        print("Found at index:", idx)
        # You can now save or process this sample
        rgb_tensor = sample[0][:,0,:,:].permute(1,2,0)  # shape [3,128,128] -> [128,128,3]
        ppg_signal = sample[1]
        
        #saving
        np.save('temp2_ppg.npy', ppg_signal.cpu().numpy())
        np.save('temp2_rgb.npy', rgb_tensor.cpu().numpy())

        #plotting
        plt.imsave('temp_frame2.png', rgb_tensor.cpu().numpy())
        plt.clf()
        plt.plot(ppg_signal)
        plt.savefig('temp2_ppg')
        break

Found at index: 1268
Found at index: 898


In [51]:
from scipy.signal import welch

# HR calculation based on ground truth label
def get_hr(y, sr=30, min=30, max=180):
    p, q = welch(y, sr, nfft=1e5/sr, nperseg=np.min((len(y)-1, 256)))
    return p[(p>min/60)&(p<max/60)][np.argmax(q[(p>min/60)&(p<max/60)])]*60

In [52]:
arr = np.load('temp1_ppg.npy')
get_hr(arr)

np.float64(30.243024302430243)

In [58]:

# Load the signal
arr = np.load('temp1_ppg.npy')
arr = arr - np.mean(arr)  # Remove DC offset

# Compute FFT
fft_vals = np.fft.fft(arr)
fft_freqs = np.fft.fftfreq(len(arr), d=1/30)  # Adjust `d` to match your sampling interval

# Take magnitude (absolute value) for plotting
fft_magnitude = np.abs(fft_vals)

# Plot only the positive frequencies
half = len(arr) // 2
plt.clf()
plt.plot(fft_freqs[:half], fft_magnitude[:half])
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("FFT of PPG Signal")
plt.grid(True)
plt.savefig('temp_fft')
